In [ ]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#  Copyright (c) 2021. Mohamed Reda Bouadjenek, Deakin University              +
#           Email:  reda.bouadjenek@deakin.edu.au                              +
#                                                                              +
#  Licensed under the Apache License, Version 2.0 (the "License");             +
#   you may not use this file except in compliance with the License.           +
#    You may obtain a copy of the License at:                                  +
#                                                                              +
#                 http://www.apache.org/licenses/LICENSE-2.0                   +
#                                                                              +
#    Unless required by applicable law or agreed to in writing, software       +
#    distributed under the License is distributed on an "AS IS" BASIS,         +
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  +
#    See the License for the specific language governing permissions and       +
#    limitations under the License.                                            +
#                                                                              +
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [ ]:
# To be run on Google Colab!
# !mkdir preprocessing/
# !wget --directory-prefix=preprocessing/  https://raw.githubusercontent.com/rbouadjenek/DeepQA/master/DeepQA/preprocessing/__init__.py   > /dev/null 2> /dev/null 
# !wget --directory-prefix=preprocessing/ https://raw.githubusercontent.com/rbouadjenek/DeepQA/master/DeepQA/preprocessing/preprocessing.py  > /dev/null 2> /dev/null 
# !pip install tokenizers transformers keras_metrics > /dev/null 2> /dev/null 

# from google.colab import drive
# drive.mount('/content/drive')

use_tpu = False


In [ ]:
import os
import random
import json
import tarfile
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import backend as K
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, BertConfig, TFBertModel
import tensorflow as tf
from tensorflow.keras import layers
from preprocessing.preprocessing import create_squad_examples, create_inputs_targets, get_SQuAD1, get_SQuAD2, get_NewsQA

# Set the random seeds
np.random.seed(1)
random.seed(1)
tf.random.set_seed(1)




# Data preprocessing

In [ ]:
# set max length value
max_len = 300
# Create the tokenizer
save_path = os.path.expanduser("~") + "/.bert_base_uncased/"
if not os.path.exists(save_path):
    slow_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    os.makedirs(save_path)
    slow_tokenizer.save_pretrained(save_path)
tokenizer = BertWordPieceTokenizer(save_path + "vocab.txt", lowercase=True)


In [ ]:
# Get the raw data

# raw_train_data, raw_val_data, raw_test_data = get_SQuAD1()
raw_train_data, raw_val_data, raw_test_data = get_SQuAD2()
# raw_train_data, raw_val_data, raw_test_data = get_NewsQA()

In [ ]:
# Prepare the data

train_squad_examples, train_skipped_questions = create_squad_examples(raw_train_data, max_len, tokenizer, False)
x_train, y1_train, y2_train, y3_train, train_total_skipped_questions = create_inputs_targets(train_squad_examples)
print('Size: ', len(y1_train[0]))
print('skipped_questions: ', train_skipped_questions)
print('skipped_questions: ', train_total_skipped_questions)

eval_squad_examples, val_skipped_questions = create_squad_examples(raw_val_data, max_len, tokenizer)
x_eval, y1_eval, y2_eval, y3_eval, val_total_skipped_questions = create_inputs_targets(eval_squad_examples)
print('Size: ', len(y1_eval[0]))
print('skipped_questions: ', val_skipped_questions)
print('skipped_questions: ', val_total_skipped_questions)

test_squad_examples, test_skipped_questions = create_squad_examples(raw_test_data, max_len, tokenizer)
x_test, y1_test, y2_test, y3_test, test_total_skipped_questions = create_inputs_targets(test_squad_examples)
print('Size: ', len(y1_test[0]))
print('skipped_questions: ', test_skipped_questions)
print('skipped_questions: ', test_total_skipped_questions)

# Create the keras model 

In [ ]:
# Define the f1-score measure to be used during training

def recall_m(y_true, y_pred):     
    true_positives = K.sum(K.round(K.clip(y_true[:,1:] * y_pred[:,1:], 0, 1)), axis=1) + 1 
    possible_positives = K.sum(K.round(K.clip(y_true[:,1:], 0, 1)), axis=1) + 1
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives =  K.sum(K.round(K.clip(y_true[:,1:] * y_pred[:,1:], 0, 1)), axis=1) + 1
    predicted_positives = K.sum(K.round(K.clip(y_pred[:,1:], 0, 1)), axis=1) + 1
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)    
    recall = recall_m(y_true, y_pred)
    f1_score = (2*precision*recall)/(precision+recall+K.epsilon())
    return K.mean(f1_score)


def exact_match(y_true, y_pred):
    clf_positives = K.sum(K.round(y_pred[:,1:]), axis=1)
    possible_positives = K.sum(K.round(K.clip(y_true[:,1:], 0, 1)), axis=1)
    accuracy_boolean = clf_positives == possible_positives
    accuracy = K.cast(accuracy_boolean, K.floatx())
    return  accuracy

def answerability_accuracy(y_true, y_pred):     
    return K.cast(K.equal(K.round(K.clip(y_pred[:,0], 0, 1)),
                          y_true[:,0]),
                          K.floatx()) 


In [ ]:
# Define the model
def create_model(learn_rate=5e-5, dropout_prob=0.3):
    ## BERT encoder 
    configuration = BertConfig(hidden_dropout_prob=dropout_prob, attention_probs_dropout_prob=dropout_prob) 
    encoder = TFBertModel.from_pretrained("bert-base-uncased", config=configuration)
    ## QA Model
    input_ids = layers.Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    token_type_ids = layers.Input(shape=(max_len,), dtype=tf.int32, name="token_type_ids")
    attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32, name="attention_mask") 
    
    embedding = encoder(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)[0]
    
    out = layers.Dense(1, name="logit", use_bias=False)(embedding)
    out = layers.Flatten()(out)
    out = layers.Dropout(dropout_prob)(out)
    out = layers.Activation(keras.activations.sigmoid)(out)
    model = keras.Model(
        inputs=[input_ids, token_type_ids, attention_mask],
        outputs=[out],
    )
    loss = keras.losses.BinaryCrossentropy(from_logits=False)
    optimizer = keras.optimizers.Adam(lr=learn_rate)
    model.compile(optimizer=optimizer, loss=[loss], metrics=[exact_match, answerability_accuracy, f1_m, precision_m, recall_m], 
                  run_eagerly=False) 
    print(model.summary())
    return model


In [ ]:
if use_tpu:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    

# Start the training

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

for lr in [9e-06, 1e-05, 3e-05, 5e-05, 7e-05, 9e-05, 1e-04]:
    for dropout_prob in [0, 0.2, 0.4, 0.6]:
        print("Learning rate = " + str(lr) + ", dropout_prob = " + str(dropout_prob))
        np.random.seed(1)
        random.seed(1)
        tf.random.set_seed(1)
        if use_tpu: 
            with strategy.scope():
                model = create_model(lr, dropout_prob)
        else:
            model = create_model(lr, dropout_prob) 
        history = model.fit(
            x_train,
            y2_train,
            validation_data=(x_eval,y2_eval),
            epochs=20, 
            verbose=1,
            batch_size=128,
            callbacks=[callback],
        )
        results = model.evaluate(x_eval, y2_eval, batch_size=128)
        print(f"Results validation: {results}, Learning rate =  {lr}, dropout_prob = {dropout_prob}")
        print('-'*50)


# Run the model with the best parameters

In [ ]:
# Hyper-parameters
# SQuAD2.0
# file_name = "SQuAD2.0_"
# lr = 5e-05
# dropout_prob = 0.2

# NewsQA
# file_name = "NewsQA_"
# lr = 9E-06
# dropout_prob = 0.2


In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
np.random.seed(1)
random.seed(1)
tf.random.set_seed(1)
if use_tpu: 
    with strategy.scope():
        model = create_model(lr, dropout_prob)

history = model.fit(
    x_train,
    y2_train,
    validation_data=(x_eval,y2_eval),
    epochs=20, 
    verbose=1,
    batch_size=128,
    callbacks=[callback],
)
model.evaluate(x_test, y2_test, batch_size=128)


In [ ]:
acc = history.history['exact_match']
val_acc = history.history['val_exact_match']
loss = history.history['loss']
val_loss = history.history['val_loss']
f1 = history.history['f1_m']
val_f1 = history.history['val_f1_m']
epochs = range(1, len(acc) + 1)

plt.figure(figsize=(15,4))

ax1 = plt.subplot(1, 3, 1)
plt.plot(epochs, loss, label='Training loss')
plt.plot(epochs, val_loss, label='Validation loss')
plt.fill_between(epochs, loss,val_loss,color='g',alpha=.1)
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

ax2 = plt.subplot(1, 3, 2)
plt.plot(epochs, acc, label='Training accuracy')
plt.plot(epochs, val_acc, label='Validation accuracy')
plt.fill_between(epochs, acc,val_acc,color='g',alpha=.1)
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

ax3 = plt.subplot(1, 3, 3)
plt.plot(epochs, f1, label='Training F1-Score')
plt.plot(epochs, val_f1, label='Validation F1-Score')
plt.fill_between(epochs, f1,val_f1,color='g',alpha=.1)
plt.title('Training and validation F1-Score')
plt.xlabel('Epochs')
plt.ylabel('F1-Score')
plt.legend()
plt.show()

In [ ]:
test_pred = model.predict(x_test, verbose=1)
eval_pred = model.predict(x_eval, verbose=1)


In [ ]:
# Zip and copy the results

algorithm = "milp_"
file_list = []

def save_to_file(name, data):
    json_string = json.dumps(data.tolist())
    file = algorithm + file_name + name
    file_list.append(file)
    with open(file, "w") as text_file:
        text_file.write(json_string)

save_to_file("test_pred.txt", test_pred)
save_to_file("eval_pred.txt", eval_pred)

save_to_file("y2_eval.txt", y2_eval)
save_to_file("y3_eval.txt", np.array([v.tolist() for v in y3_eval]))

save_to_file("y2_test.txt", y2_test)
save_to_file("y3_test.txt", np.array([v.tolist() for v in y3_test]))

save_to_file("token_type_eval.txt", x_eval[1])
save_to_file("token_type_test.txt", x_test[1])

save_to_file("input_ids_test.txt", x_test[0])


tar = tarfile.open(algorithm + file_name + "predictions.tar.gz", "w:gz")
for name in file_list:
    tar.add(name)
tar.close()

!mv *predictions.tar.gz /content/drive/MyDrive


# Debugging

In [ ]:
np.random.seed(1)
random.seed(1)
tf.random.set_seed(1)
model = create_model(5e-05, 0) 
i = 16
history = model.fit(
        [x_test[0][0:i],x_test[1][0:i],x_test[2][0:i]],
        y2_test[0:i],
        epochs=2, 
        verbose=1,
        batch_size=4,
    )
model.evaluate([x_test[0][0:i],x_test[1][0:i],x_test[2][0:i]],y2_test[0:i], batch_size=4)
